# S2M2

## Integrating Omics data into *Genome-Scale Metabolic Models (GEMs)* with *Troppo*


High-throughput data from various omics fields can be integrated into *GEMs* to improve phenotype prediction. This approach aims to either further constrain the solution space or create context-specific models from generic *GEMs*.
When considering omics data to integrate into models, transcriptomics appears as the most used. Transcript profiles describe the expression under specific conditions for all known genes of the studied organism. Moreover, it is the most accessible to obtain experimentally and already has well-established techniques, such as microarrays and RNA-seq.

## *Troppo*

Troppo (Tissue-specific RecOnstruction and Phenotype Prediction using Omics data) is a Python package containing methods for tissue specific reconstruction to use with constraint-based models.

The current methods implemented are:
- FastCORE
- CORDA
- GIMME
- (t)INIT
- iMAT

# How to use *GIMME* with *Troppo*

A typical workflow follows two main steps. The first is to attribute a score to each reaction of the model, in accordance with the omics data imputed. The second is to use the scores and apply an integration method to select a subset of reactions to build the final model.

Integration scoring methods implemented in *Troppo* are:
- continuous: `ContinuousScoreIntegrationStrategy`
- threshold: `ThresholdSelectionIntegrationStrategy`
- default_core: `DefaultCoreIntegrationStrategy`
- adjusted_score: `AdjustedScoreIntegrationStrategy`
- custom: `CustomSelectionIntegrationStrategy`

Omics integration methods implemented in *Troppo* are:
- gimme: `GIMME`
- tinit: `tINIT`
- fastcore: `GIMME`
- imat: `IMAT`
- swiftcore: `SWIFTCORE`
- corda: `CORDA`

This example can be applied for all the Omics integration methods implemented in this package. Note that the appropriate integration scoring method can differ between integration algorithms. For instance, for *GIMME* a continuous scoring method can be used, while for `fastcore` a threshold scoring method is more adequate.

### Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy, CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.gimme import GIMME, GIMMEProperties
from cobamp.utilities.parallel import batch_run
from copy import deepcopy

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\wrappers\external_wrappers.py:12: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  reader can also be created using the get_model_reader function on cobamp.wrappers''')


### Initial Setup

Define the parsing rules for the GPRs that will be used later on.

In [2]:
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

#### Read model and omics data

You have to download the .csv file containing the gene expression for breast cancer cell lines present in the CCLE panel. The nomenclature has been normalized for what is found in the metabolic model of this exercise.

In [3]:
model = cobra.io.load_matlab_model('data/redHUMAN_recon2_smin.mat')
model

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model redHUMAN_recon2_smin_02Sep2019_135437. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, l, m, n, r, x


Name,redHUMAN_recon2_smin_02Sep2019_135437
Memory address,1923e805f48
Number of metabolites,469
Number of reactions,1396
Number of genes,699
Number of groups,49
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, m, x, e, r, l, n"


For future usage, note that the dataset that will be used as input in *Troppo* needs to have the samples as rows and gene IDs in the columns.

In [5]:
omics_data = pd.read_csv('data/CCLE_breast_cancer_expression.csv', index_col=0)
omics_data.shape

(56, 579)

### Create a container for the omics data.

The `TabularReader` class is used to read and store the omics data in a container that can then be used by *Troppo*. 

Relevant arguments from the `TabularReader` class:
- `path_or_df`: the omics data can be either a pandas dataframe or a path to a dataset file. The file can be in any format supported by pandas.
- `index_col`: the name of the column that contains the identifiers of the genes.
- `sample_in_rows`: a boolean indicating whether the samples are in rows or columns.
- `header_offset`: the number of rows to skip before reading the header.
- `omics_type`: a string containing the type of omics data. This is used to select the appropriate integration method.
- `nomenclature`: a string containing the nomenclature of the identifiers in the omics data. This is used to map the identifiers to the identifiers in the model.

The `to_containers()` method returns a list of containers, one for each sample of the dataset. In this example, we will be using only one sample, however, the process can be iterated for all the samples in the dataset.
The `get_integrated_data_map()` method is used to map the identifiers in the omics data to the identifiers in the model. This is done by using the `gpr_gene_parse_function` argument from the `ModelBasedWrapper` class.

In [6]:
omics_container = TabularReader(path_or_df=omics_data, nomenclature='entrez_id', omics_type='transcriptomics').to_containers()
omics_container

In [7]:
single_sample = omics_container[0]

In [8]:
print(single_sample)

transcriptomics container
Condition: ACH-000017
Nomenclature: entrez_id
579 Expression Values


### Create a model wrapper.

The `ModelBasedWrapper` class is used to wrap the model so that it can be used by *Troppo*.

Relevant arguments from this class include:
- `model`: the model to be wrapped.
- `ttg_ratio`: the ratio between the number of reactions to be selected and the total number of reactions in the model.
- `gpr_gene_parse_function`: a function that parses the GPRs of the model. This is used to map the identifiers in the omics data to the identifiers in the model.

Important attributes from this class include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model, such as, reaction_ids, metabolite_ids, GPRs, bounds, etc.
- `S`: the stoichiometric matrix of the model.
- `lb`: the lower bounds of the reactions in the model.
- `ub`: the upper bounds of the reactions in the model.

In this specific example we will use the `ReconstructionWrapper` class instead of the base `ModelBasedWrapper` class.

In [9]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999, gpr_gene_parse_function=replace_alt_transcripts)
model_wrapper

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _10993
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _89874
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _57835 or _8671 or _57835 or _57835 or _57835
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: (_4967 and (_1738 and _8050) and _1743) or (_4967 and (_1738 and _8050) and _1743)
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _3242
  warnings.warn('Co

### Map the identifiers in the omics data to the identifiers in the model

For this we can use the `get_integrated_data_map()` method from the `TabularReader` class. This maps the gene ids in the omics dataset reaction ids in the model through their GPRs, and attributes a score to each reaction in accordance with the expression values of the associated genes. This method returns a dictionary with the reaction ids as keys and the scores as values.

Important arguments from this method include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model. It can be accessed through the `model_wrapper.model_reader`.
- `and_func`: a function that is used to combine the scores of the genes associated with a reaction for AND rules in the GPR. In this example, we will be using the minimum function, which means that the score of a reaction with AND in their GPRs will be the minimum score of the genes associated with it.
- `or_func`: a function that is used to combine the scores of the genes associated with a reaction for OR rules in the GPR. In this example, we will be using the sum function, which means that the score of a reaction with OR in their GPRs will be the sum of the scores of the genes associated with it.

In [10]:
data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader, and_func=min, or_func=sum)

### Integrate scores

The `integrate()` method from the `ContinuousScoreIntegrationStrategy` class is used to integrate the scores of the reactions in the model. This method returns a dictionary with the reaction ids as keys and the integrated scores as values. In the case of this continuous scoring method, the resulting scores are the same as the scores in the data map. However, for other scoring methods, such as threshold scoring methods, the result will be a list of reactions with a score above the selected threshold. 

Moreover, this method allows us to apply an additional function to the method, which can be useful if you have any protected reactions that need to be in the final model or to remove nan values from the result. This can be done by passing the function as the `score_apply` argument of the `ContinuousScoreIntegrationStrategy` class. 

In this example, we will be using a function that replaces the nan values with 0 and returns a list with all the scores. This is the required format for the *GIMME* method.

In [11]:
def score_apply(reaction_map_scores):
    return {k:0  if v is None else v for k, v in reaction_map_scores.items()}

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
scores = continuous_integration.integrate(data_map=data_map)

print(scores)

{'2AMACHYD': 0.475084882949, '2AMADPTm': 0.150559676575, '2DR1PP': 0, '2HCO3_NAt': 2.6904623550171003, '2OXOADOXm': 9.76137105052, '2OXOADPTm': 0.150559676575, '34HPPOR': 0.0143552929771, '3AIBt': 0, '3AIBTm': 1.496922466008, '3AIBtm': 1.496922466008, '3HAO': 0.650764559117, '3HBCOAHLm': 0, '3HKYNAKGAT': 2.9180564378900007, '3MOBt2im': 0, '3MOPt2im': 0, '3SALAASPm': 9.261276459529999, '3SALATAi': 5.09254574154, '3SALATAim': 6.91683488515, '3SPYRSP': 0, '3SPYRSPm': 0, '4ABUTtm': 0, '4MOPt2im': 0, '5MTHFt': 15.807559610700002, '5MTHFt2': 9.09317165802, 'AACOAT': 4.80322703643, 'AACTOOR': 0.111031312389, 'AACTtm': 0, 'AASAD3m': 0, 'AATAi': 1.3919876262200002, 'ABTArm': 1.496922466008, 'ABUTD': 5.591559745830001, 'ABUTt2r': 3.0478873294, 'ABUTt4_2_r': 0.20769798096859998, 'ACACT10m': 18.406597148190002, 'ACACT1r': 4.66106547981, 'ACACt2': 0.9456077031814, 'ACACt2m': 0.8479969065549999, 'ACETONEt2': 8.835628723441399, 'ACETONEt2m': 0.8479969065549999, 'ACITL': 15.32896568072, 'ACOAD10m': 7.

Below is and example on how to use the `CustomSelectionIntegrationStrategy` class to integrate the scores of the reactions in the model. This is basically a threshold scoring method that allows us to also keep a set of protected reactions. This will output a list of core reactions that will be used to build the final model with the *FastCORE* method.

In [12]:
from math import log
threshold =  (5 * log(2))
protected_reactions = ['biomass']

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)

print(threshold_scores)

[['2OXOADOXm', '3SALAASPm', '3SALATAi', '3SALATAim', '5MTHFt', '5MTHFt2', 'AACOAT', 'ABUTD', 'ACACT10m', 'ACACT1r', 'ACETONEt2', 'ACITL', 'ACOAD10m', 'ACOAD9m', 'ACONT', 'ACONTm', 'ACS', 'ACYP', 'ADEt', 'ADK1', 'ADK1m', 'ADMDC', 'ADNK1', 'ADPT', 'AHC', 'AKGDm', 'AKGMALtm', 'ALAASNNaEx', 'ALACYSNaEx', 'ALAGLNNaEx', 'ALASERNaEx', 'ALAt4', 'ALATA_L', 'ALATHRNaEx', 'AMETt2m', 'ARGLYSex', 'ARGtiDF', 'ARGtm', 'ASNALANaEx', 'ASNCYSNaEx', 'ASNGLNNaEx', 'ASNS1', 'ASNSERNaEx', 'ASNt4', 'ASNTHRNaEx', 'ASPGLUm', 'ASPTA', 'ASPTAm', 'ATPS4m', 'ATPtm', 'CATm', 'CATp', 'CBPS', 'CBPSam', 'CHOLtu', 'CHSTEROLt1', 'CITRtm', 'CITtam', 'CITtbm', 'CK', 'CKc', 'CLHCO3tex2', 'CLOHtex2', 'CLOXAtex2', 'COAtm', 'CREATt4_2_r', 'CSm', 'CTPS1', 'CTPS2', 'CYOR_u10m', 'CYSALANaEx', 'CYSASNNaEx', 'CYSGLNNaEx', 'CYSGLUexR', 'CYSGLYexR', 'CYSSERNaEx', 'CYSt4', 'CYStec', 'CYSTGL', 'CYSTHRNaEx', 'DNDPt11m', 'DNDPt23m', 'DNDPt24m', 'DNDPt34m', 'DNDPt35m', 'DNDPt9m', 'DPGase', 'DPGM', 'ECOAH12m', 'ECOAH1m', 'ECOAH9m', 'ENO',

### Run the GIMME algorithm

The `GIMMEProperties` class is used to create the properties for the GIMME algorithm. This class contains the following arguments:
- `exp_vector`: a list of scores for each reaction in the model. This can be obtained from the `integrate()` method of the `ContinuousScoreIntegrationStrategy` class.
- `objectives`: a list of dictionaries with the reactions to be used as objectives. Each dictionary should have the reaction id as key and the coefficient as value.
- `preprocess`: a boolean indicating if the model should be preprocessed before running the GIMME algorithm. This is useful if you want to remove reactions that are not connected to the biomass reaction.
- `flux_threshold`: a threshold to remove reactions with fluxes below it. This is useful if you want to remove reactions that are not connected to the biomass reaction.
- `obj_frac`: the flux fraction of the objective reactions to be used.

The `GIMME` class is used to run the GIMME algorithm. This class contains the following arguments:
- `S`: the stoichiometric matrix of the model. It can be accessed through the `model_wrapper.S`.
- `lb`: the lower bounds of the reactions in the model. It can be accessed through the `model_wrapper.lb`.
- `ub`: the upper bounds of the reactions in the model. It can be accessed through the `model_wrapper.ub`.
- `properties`: a `GIMMEProperties` instance containing the properties for the GIMME algorithm.

In the end, the `run()` method of the `GIMME` class will return a list of the active reactions index that should be keep in the final model.

In [14]:
# Get the index of the biomass reaction in the model. This will be used as objective for the GIMME algorithm.
idx_objective = model_wrapper.model_reader.r_ids.index('biomass')

# Create the properties for the GIMME algorithm.
properties = GIMMEProperties(exp_vector=[v for k, v in scores.items()], obj_frac=0.8, objectives=[{idx_objective: 1}],
                             preprocess=True, flux_threshold=0.8, solver='GLPK', 
                             reaction_ids= model_wrapper.model_reader.r_ids, metabolite_ids=model_wrapper.model_reader.m_ids)

# Run the GIMME algorithm.
gimme = GIMME(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

gimme_run = gimme.run()
gimme_run

Could not set parameters with this solver
Could not set parameters with this solver
Could not set parameters with this solver


[3,
 4,
 8,
 9,
 12,
 15,
 16,
 17,
 21,
 22,
 23,
 24,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 46,
 51,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 63,
 64,
 65,
 66,
 67,
 68,
 70,
 72,
 73,
 74,
 75,
 76,
 79,
 82,
 83,
 84,
 85,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 99,
 101,
 102,
 108,
 109,
 110,
 111,
 115,
 117,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 128,
 129,
 130,
 132,
 133,
 134,
 135,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 147,
 148,
 149,
 150,
 151,
 152,
 155,
 156,
 157,
 158,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 186,
 187,
 188,
 189,
 195,
 196,
 198,
 202,
 205,
 207,
 209,
 210,
 211,
 214,
 217,
 218,
 219,
 221,
 223,
 224,
 225,
 226,
 227,
 228,
 230,
 231,
 232,
 233,
 234,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 256,
 257,
 264,
 265,
 266,
 267,
 269,
 270,
 271,
 274,
 2

Moreover, to access the flux distribution determined by the algorithm, you can use the `sol` attribute of the `GIMME` class.

In [15]:
gimme.sol.__dict__['_Solution__value_map']

OrderedDict([('2AMACHYD', 0.0),
             ('2AMADPTm', 0.0),
             ('2DR1PP', 0.0),
             ('2HCO3_NAt', 0.0),
             ('2OXOADOXm', 0.0),
             ('2OXOADPTm', 0.0),
             ('34HPPOR', 0.0),
             ('3AIBt', 0.0),
             ('3AIBTm', 0.0),
             ('3AIBtm', 0.0),
             ('3HAO', 0.0),
             ('3HBCOAHLm', 0.0),
             ('3HKYNAKGAT', 0.0),
             ('3MOBt2im', 0.0),
             ('3MOPt2im', 0.0),
             ('3SALAASPm', 0.0),
             ('3SALATAi', 0.0),
             ('3SALATAim', 0.0),
             ('3SPYRSP', 0.0),
             ('3SPYRSPm', 0.0),
             ('4ABUTtm', 0.0),
             ('4MOPt2im', 0.003463076480964108),
             ('5MTHFt', 0.0),
             ('5MTHFt2', 0.0),
             ('AACOAT', 0.0),
             ('AACTOOR', 0.0),
             ('AACTtm', 0.0),
             ('AASAD3m', 0.0),
             ('AATAi', 0.0),
             ('ABTArm', 0.0),
             ('ABUTD', 0.0),
             ('A

Furthermore, is possible to reconstruct the model based on the integration results.

In [16]:
def sbml_model_reconstruction(model_template: cobra.Model, results):
    """
    This function is used to reconstruct the model based on the integration results.

    Parameters
    ----------
    model_template: cobra.Model
        The COBRA model template.
    sample: str
        The sample name.
    integration_result_dict: dict
        The integration results.
    """
    temp_model = deepcopy(model_template)

    temp_model.objective = 'biomass'  

    if isinstance(results, list):
        
        reactions_to_deactivate = [reaction for reaction in temp_model.reactions if reaction not in [temp_model.reactions[reaction_idx] for reaction_idx in results] if reaction.id != 'biomass']
        
        temp_model.remove_reactions(reactions_to_deactivate, remove_orphans=True)
        
    if isinstance(results, dict):
        
        reactions_to_deactivate = [reaction for reaction, value in results.items() if value is False if reaction != 'biomass']
                
        temp_model.remove_reactions(reactions_to_deactivate, remove_orphans=True)
        
    temp_model.id = 'Reconstructed Model'
    print('Model Objective:', temp_model.objective)
    print('Objective Production:', temp_model.optimize())
    print(f'Model reconstruction finished.')
    
    return temp_model

In [17]:
reconstructed_model = sbml_model_reconstruction(model, gimme_run)
reconstructed_model

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Model Objective: Maximize
1.0*biomass - 1.0*biomass_reverse_01e59
Objective Production: <Solution 0.035 at 0x19235919a88>
Model reconstruction finished.


Name,Reconstructed Model
Memory address,19231336408
Number of metabolites,394
Number of reactions,955
Number of genes,626
Number of groups,49
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, m, x, e, r, l, n"


Besides the Gimme class above demonstrated, Troppo allows a more direct approach to run the integration methods using `run_from_omics` method from the `ReconstructionWrapper` class.
Below is an example on how to use the `run_from_omics` method for the GIMME algorithm.

In [18]:
parameters = {'threshold': threshold, 'reconstruction_wrapper': model_wrapper, 'algorithm': 'gimme'}

In [21]:
def reconstruction_function_gimme(omics_container, parameters: dict):

    def score_apply(reaction_map_scores):
        return {k:0  if v is None else v for k, v in reaction_map_scores.items()}
    
    threshold, rec_wrapper, method = [parameters[parameter] for parameter in
                                      ['threshold', 'reconstruction_wrapper', 'algorithm']]

    reac_ids = rec_wrapper.model_reader.r_ids
    metab_ids = rec_wrapper.model_reader.m_ids
    AND_OR_FUNCS = (min, sum)    

    if method == 'gimme':
            return rec_wrapper.run_from_omics(omics_data=omics_container, algorithm=method, and_or_funcs=AND_OR_FUNCS,
                                              integration_strategy=('continuous', score_apply), solver='GLPK', obj_frac=0.8,
                                              objectives=[{'biomass': 1}], preprocess=True, flux_threshold=0.8,
                                              reaction_ids=reac_ids, metabolite_ids=metab_ids)

In [22]:
reconstruction_function_gimme(single_sample, parameters)

Could not set parameters with this solver
Could not set parameters with this solver
Could not set parameters with this solver


{'2AMACHYD': False,
 '2AMADPTm': False,
 '2DR1PP': False,
 '2HCO3_NAt': True,
 '2OXOADOXm': True,
 '2OXOADPTm': False,
 '34HPPOR': False,
 '3AIBt': False,
 '3AIBTm': True,
 '3AIBtm': True,
 '3HAO': False,
 '3HBCOAHLm': False,
 '3HKYNAKGAT': True,
 '3MOBt2im': False,
 '3MOPt2im': False,
 '3SALAASPm': True,
 '3SALATAi': True,
 '3SALATAim': True,
 '3SPYRSP': False,
 '3SPYRSPm': False,
 '4ABUTtm': False,
 '4MOPt2im': True,
 '5MTHFt': True,
 '5MTHFt2': True,
 'AACOAT': True,
 'AACTOOR': False,
 'AACTtm': False,
 'AASAD3m': False,
 'AATAi': True,
 'ABTArm': True,
 'ABUTD': True,
 'ABUTt2r': True,
 'ABUTt4_2_r': False,
 'ACACT10m': True,
 'ACACT1r': True,
 'ACACt2': True,
 'ACACt2m': True,
 'ACETONEt2': True,
 'ACETONEt2m': True,
 'ACITL': True,
 'ACOAD10m': True,
 'ACOAD9m': True,
 'ACOAHi': False,
 'ACONT': True,
 'ACONTm': True,
 'ACS': True,
 'ACSm': True,
 'ACt2m': False,
 'ACt2r': False,
 'ACTLMO': False,
 'ACTNMO': False,
 'ACYP': True,
 'ADCim': False,
 'ADEt': True,
 'ADK1': True,
 '

# Run in Batch

Using the reconstruction_function_gimme defined above, we can run the GIMME algorithm in batch for multiple samples. Below is and example on how to use the batch_run function for parallel reconstruction of the models. 
In this case we will use some samples from omics_container created previously.

In [23]:
batch_container = omics_container[0:3]
batch_container

In [24]:
[c.condition for c in batch_container]

['ACH-000017', 'ACH-000019', 'ACH-000028']

In [25]:
batch_gimme_res = batch_run(reconstruction_function_gimme, batch_container, parameters, threads=10)

In [26]:
results = {c.condition: res for c, res in zip(batch_container, batch_gimme_res)}
results

{'ACH-000017': {'2AMACHYD': False,
  '2AMADPTm': False,
  '2DR1PP': False,
  '2HCO3_NAt': True,
  '2OXOADOXm': True,
  '2OXOADPTm': False,
  '34HPPOR': False,
  '3AIBt': False,
  '3AIBTm': True,
  '3AIBtm': True,
  '3HAO': False,
  '3HBCOAHLm': False,
  '3HKYNAKGAT': True,
  '3MOBt2im': False,
  '3MOPt2im': False,
  '3SALAASPm': True,
  '3SALATAi': True,
  '3SALATAim': True,
  '3SPYRSP': False,
  '3SPYRSPm': False,
  '4ABUTtm': False,
  '4MOPt2im': True,
  '5MTHFt': True,
  '5MTHFt2': True,
  'AACOAT': True,
  'AACTOOR': False,
  'AACTtm': False,
  'AASAD3m': False,
  'AATAi': True,
  'ABTArm': True,
  'ABUTD': True,
  'ABUTt2r': True,
  'ABUTt4_2_r': False,
  'ACACT10m': True,
  'ACACT1r': True,
  'ACACt2': True,
  'ACACt2m': True,
  'ACETONEt2': True,
  'ACETONEt2m': True,
  'ACITL': True,
  'ACOAD10m': True,
  'ACOAD9m': True,
  'ACOAHi': False,
  'ACONT': True,
  'ACONTm': True,
  'ACS': True,
  'ACSm': True,
  'ACt2m': False,
  'ACt2r': False,
  'ACTLMO': False,
  'ACTNMO': False,

In [27]:
reconstructed_models = {c: sbml_model_reconstruction(model, res) for c, res in results.items()}
reconstructed_models

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Model Objective: Maximize
1.0*biomass - 1.0*biomass_reverse_01e59
Objective Production: <Solution 0.035 at 0x19236605508>
Model reconstruction finished.
Model Objective: Maximize
1.0*biomass - 1.0*biomass_reverse_01e59
Objective Production: <Solution 0.035 at 0x192325c0ec8>
Model reconstruction finished.
Model Objective: Maximize
1.0*biomass - 1.0*biomass_reverse_01e59
Objective Production: <Solution 0.035 at 0x19234743ec8>
Model reconstruction finished.


{'ACH-000017': <Model Reconstructed Model at 0x192382e8688>,
 'ACH-000019': <Model Reconstructed Model at 0x19238022c48>,
 'ACH-000028': <Model Reconstructed Model at 0x192325cb548>}